```load json doc```

In [1]:
import json 

with open('./documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

``` import model for vectorizing```

In [2]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


```check does it works or not```

In [4]:
res = model.encode('Timur')

In [5]:
len(res)

384

```indexes text and vector representation both```

In [9]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

```vectorizing question, text and both question+text and add it to full documents```

In [11]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|█████████████████████████████████████████████████████████████████████████| 1007/1007 [02:03<00:00,  8.18it/s]


In [12]:
documents[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'id': '1f6520ca',
 'question_vector': array([ 3.37158665e-02,  6.56111315e-02,  5.33279148e-04, -1.34431536e-03,
        -3.15999836e-02, -2.65931501e-03, -2.74103973e-03, -1.58601347e-02,
        -1.31478801e-01,  8.15986544e-02, -4.36173491e-02, -5.51428646e-02,
         7.84663297e-03,  3.35141085e-02, -3.39554735e-02, -4.22502011e-02,
         5.02886856e-03, -4.80887340e-03,  5.66817969e-02, -5.18101491e-02,
        -4.96364571e-03,  2.41137184e-02,  1.30297076e-02, -2.91536562e-02,
         4.07025367e-02,  7.36691570e-03,  4.59836051e-02,  4.69923392e-02,
        -6.24345010e-03,  3.77092138e-02, -2.48126667e-02, -1.76821388e-02,
         8.72684568e-02,  1.79800652e-02, -5.46353906e-02,  6.17317706e-02,
         2.20844056e-02, -4.70293425e-02,  1.8

```indexes our ElasticSearch```

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████| 1007/1007 [00:21<00:00, 47.14it/s]


```vectorizing our query for furhter query search```

In [14]:
query = 'I just discovered the course. Can I still join it?'

v_q = model.encode(query)

```Try to use vector search in our ElasticSearch```

In [15]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

#### Real example with our dataset

In [18]:
import pandas as pd

df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')


In [19]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

```Set metrics```

##### Hit rate

In [23]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

##### MRR

In [24]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

##### Function for evaluating our search engine

In [21]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

```Evaluation question vector search```

In [25]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

evaluate(ground_truth, question_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████| 4627/4627 [01:39<00:00, 46.51it/s]


{'hit_rate': 0.7581586341041712, 'mrr': 0.6536092500540314}

```Evaluation text vector search```

In [26]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

evaluate(ground_truth, text_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████| 4627/4627 [01:38<00:00, 46.79it/s]


{'hit_rate': 0.8121893235357683, 'mrr': 0.6903789352352149}

```Evaluation comdined question_text vector search```

In [ ]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

evaluate(ground_truth, question_text_vector_knn)

 75%|██████████████████████████████████████████████████████▉                  | 3486/4627 [01:14<00:24, 47.20it/s]

```Evaluating combined vector search```

In [27]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn)